In [14]:
!pip install -q --upgrade langchain_community

In [19]:
!pip install tenacity==8.3.0

  Using cached tenacity-8.3.0-py3-none-any.whl.metadata (1.2 kB)
Using cached tenacity-8.3.0-py3-none-any.whl (25 kB)
  Attempting uninstall: tenacity
    Found existing installation: tenacity 8.4.0
    Uninstalling tenacity-8.4.0:
      Successfully uninstalled tenacity-8.4.0


In [20]:
from langchain_community.document_loaders import WebBaseLoader


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [23]:
loader = WebBaseLoader(["https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/"])
docs = loader.load()

In [24]:
!pip install -qU langchain-text-splitters

In [71]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
split_docs = text_splitter.split_documents(docs)

In [79]:
!pip install langchain-openai
!pip install langchain-chroma
!pip install langchainhub

In [103]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI

embeddings_model = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(documents=split_docs, embedding=embeddings_model)
retriever = vectorstore.as_retriever()

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

retriever = vectorstore.as_retriever()
query = "agent memory"
retrieve_contents = retriever.get_relevant_documents("agent memory")


In [105]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate



parser = JsonOutputParser()

prompt = PromptTemplate(
    template="is it relevance contents with user query? If yes, answer `yes` in the relevance field. If not, answer no.\ncontents: {retrieve_contents} \n{format_instructions}\n{query}\n",
    input_variables=["retrieve_contents", "query"],    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm | parser

chain.invoke({"retrieve_contents": retrieve_contents, "query": query})

{'relevance': 'no',
 'agent_memory': {'Memory_stream': "a long-term memory module that records agents' experiences in natural language",
  'Retrieval_model': "surfaces context to inform agent's behavior based on relevance, recency, and importance",
  'Reflection_mechanism': "synthesizes memories into higher level inferences over time and guides agent's future behavior"}}